# TripPal Search Engine 
this search engine provides integration between different data APIs; like `Dojo Hotel` and `OpenTripMap` 

In [3]:
import pandas as pd
import numpy as np
import requests 
import json
import math
from utils import hotels_engine as hotels
from utils import places_engine as places 
from utils import data_parser as dp
from utils import constants as c

### Section One - Hotels search engine 

In [4]:
def search_location(loc, pn, check_in, check_out, adults, sort_order = 'PRICE', filters = {}):
    '''
    sort_order : One of the following is allowed: 
    BEST_SELLER|
    STAR_RATING_HIGHEST_FIRST|
    STAR_RATING_LOWEST_FIRST|
    DISTANCE_FROM_LANDMARK|
    GUEST_RATING|
    PRICE_HIGHEST_FIRST|
    PRICE
    '''    
    idx = hotels.get_city_id(loc)
    
    res = hotels.get_hotels_by_des_id(idx, pn, check_in, check_out, adults, sort_order , filters)
    
    df = hotels.get_search_results(res)
    
#     df = dp.clean_results(df)
    
    jsn = hotels.get_search_filters(res)
    
#     return {'HOTELS' : dp.get_json_file(df),
#             'FILTERS' : jsn}
    return df

In [5]:
res = search_location('chtaura', 1, "2021-03-16", "2021-03-20", 1)

In [6]:
res.head(2)

,id,name,starRating,urls,address,guestReviews,landmarks,geoBullets,ratePlan,neighbourhood,deals,messaging,badging,pimmsAttributes,coordinate,providerType,supplierHotelId,isAlternative,optimizedThumbUrls,vrBadge
0,645771,Delora Hotel and Suites,3.5,{},"{'streetAddress': 'Damascus Road', 'extendedAd...","{'unformattedRating': 8.8, 'rating': '8.8', 't...","[{'label': 'City center', 'distance': '1.0 mil...",[],"{'price': {'current': '$80', 'exactCurrent': 8...",Chtaura,{},{},{},DoubleStamps|D13|TESCO,"{'lat': 33.817606, 'lon': 35.847565}",LOCAL,16361572,False,{'srpDesktop': 'https://thumbnails.trvl-media....,NaN
1,660643968,Massabki Hotel,4.0,{},"{'streetAddress': 'Chtaura Main Road, 1802', '...","{'unformattedRating': 8.6, 'rating': '8.6', 't...","[{'label': 'City center', 'distance': '0.8 mil...",[],"{'price': {'current': '$108', 'exactCurrent': ...",Chtaura,{},{},{},DoubleStamps|D13|TESCO,"{'lat': 33.81591, 'lon': 35.85193}",LOCAL,20613874,False,{'srpDesktop': 'https://thumbnails.trvl-media....,NaN


In [7]:
def best_hotels_in_country(country):
    return  search_location(country, 
                            1, 
                            "2020-01-01",
                            "2021-01-01", 1,
                            'STAR_RATING_HIGHEST_FIRST' ,
                            {"guestRatingMin":"8"})

In [8]:
co = best_hotels_in_country('china')

In [11]:
co.head(1)

,id,name,starRating,urls,address,guestReviews,landmarks,geoBullets,ratePlan,neighbourhood,deals,messaging,badging,pimmsAttributes,coordinate,providerType,supplierHotelId,isAlternative,optimizedThumbUrls,vrBadge
0,186370,Wanda Vista Beijing,5.0,{},"{'streetAddress': 'Tower C Wanda Plaza, 93 Jia...","{'unformattedRating': 8.6, 'rating': '8.6', 't...","[{'label': 'City center', 'distance': '3.8 mil...",[],"{'price': {'current': '$123', 'exactCurrent': ...",Jianwai Street,{},{},{},DoubleStamps|D13|TESCO,"{'lat': 39.907605, 'lon': 116.463455}",MULTISOURCE,1697569,False,{'srpDesktop': 'https://thumbnails.trvl-media....,NaN


In [16]:
def hotel_details(destination_id : int):
    
    response = hotels.get_hotel_details(destination_id)

    return dp.parse_details(response)

In [17]:
# dp.get_json_file(res)
det = hotel_details(453028)

In [18]:
det

{'HOTEL_FEATURE': ['32 smoke-free guestrooms',
  'Daily housekeeping',
  'Business center',
  'Terrace',
  '24-hour front desk',
  'Coffee/tea in a common area',
  'Air conditioning',
  'Garden',
  'Computer station',
  'Meeting room',
  'Multilingual staff',
  'Picnic area',
  'Free WiFi and free parking '],
 'LOCATION_SECTION': ['In East Williamsburg',
  'Brooklyn Bridge Park - 3.2 mi / 5.1 km',
  'Prospect Park - 3.6 mi / 5.7 km',
  'Barclays Center Brooklyn - 3.9 mi / 6.3 km',
  'Washington Square Park - 4.5 mi / 7.3 km',
  'New York University - 4.6 mi / 7.3 km',
  'Union Square Park - 4.8 mi / 7.8 km',
  'Wall Street - 4.9 mi / 8 km',
  'Chelsea Market - 5.4 mi / 8.7 km',
  'One World Trade Center - 5.5 mi / 8.8 km',
  'The High Line Park - 5.9 mi / 9.5 km'],
 'TAGLINE': ['<b>No-frills hostel in East Williamsburg </b>'],
 'HOTEL_FREEBIES': ['Free WiFi and free parking'],
 'NAME': 'NY Moore Hostel',
 'STAR_RATE': 2.0,
 'PRICE': 34.0,
 'ROOMS': ['Single Bed in Shared Dormitory, Pri

In [33]:
df = pd.DataFrame(data = det['HOTEL_FEATURE'])
df.head(10)

,0
0,32 smoke-free guestrooms
1,Daily housekeeping
2,Business center
3,Terrace
4,24-hour front desk
5,Coffee/tea in a common area
6,Air conditioning
7,Garden
8,Computer station
9,Meeting room


In [53]:
dicter = {}

In [27]:
det['HOTEL_FEATURE']

['32 smoke-free guestrooms',
 'Daily housekeeping',
 'Business center',
 'Terrace',
 '24-hour front desk',
 'Coffee/tea in a common area',
 'Air conditioning',
 'Garden',
 'Computer station',
 'Meeting room',
 'Multilingual staff',
 'Picnic area',
 'Free WiFi and free parking ']

In [54]:
res_str = ''
for row in det['HOTEL_FEATURE']:
    res_str += row
    res_str += ', '

dicter.update({'HOTEL_FEATURE' : res_str})

In [55]:
res_str = ' '
for idx in det['ROOM_FEATURES']:
    for item in idx['listItems']:
        res_str += item
        res_str += ', '
        
dicter.update({'ROOM_FEATURES' : res_str})

In [56]:
res_str = ' '
for idx in det['HOTEL_FEATURES']:
    for item in idx['listItems']:
        res_str += item
        res_str += ', '
dicter.update({'HOTEL_FEATURES' : res_str})

In [78]:
dicter.update({'NAME' : det['NAME']})

In [79]:
dfr = pd.DataFrame(data = dicter ,index = [0],
                 )

In [80]:
dfr.head()

,HOTEL_FEATURE,ROOM_FEATURES,HOTEL_FEATURES,NAME
0,"32 smoke-free guestrooms, Daily housekeeping, ...","Air conditioning, Soundproofed rooms, Individ...","Coffee/tea in a common area, Business center,...",NY Moore Hostel


In [8]:
rev = hotels.get_hotel_reviews(453028, 1)

In [9]:
imgs = hotels.get_hotel_imgs(453028)

In [10]:
idx = hotels.get_hotel_id_by_name('Albrechtshof')

In [11]:
idx

'799081408'

### Section two - Places search engine 

In [14]:
places.get_list_of_places('new yourk', 'churches')

[{'xid': 'W93406623',
  'name': "Saint Peter's Episcopal Church",
  'rate': '3h',
  'img_url': 'https://commons.wikimedia.org/wiki/File:St_Peters_Church_2011.jpg'},
 {'xid': 'W93282521',
  'name': "St Mary's Roman Catholic Church",
  'rate': '3h',
  'img_url': 'https://commons.wikimedia.org/wiki/File:StMarysChurchAlbany.jpg'},
 {'xid': 'W194233571',
  'name': 'New Jerusalem Home of the Saved Church',
  'rate': '1'},
 {'xid': 'W93282551', 'name': 'First Dutch Reformed Church', 'rate': '1'},
 {'xid': 'Q5452657',
  'name': 'First Church in Albany',
  'rate': '3h',
  'img_url': 'https://commons.wikimedia.org/wiki/File:Dutch_Church_Albany.jpg'},
 {'xid': 'W105511227',
  'name': "Saint Joseph's Park",
  'rate': '3',
  'img_url': 'https://commons.wikimedia.org/wiki/File:Saint_Josephs_Church_Albany.jpg'},
 {'xid': 'W126310802',
  'name': "Saint Joseph's Church",
  'rate': '3',
  'img_url': 'https://commons.wikimedia.org/wiki/File:Saint_Josephs_Church_Albany.jpg'},
 {'xid': 'W90702210',
  'name

In [12]:
places.get_object_properties('W331188177')

{'xid': 'W331188177',
 'name': 'Bıçakçı Alaaddin Cami',
 'address': {'city': 'İskenderpaşa Mahallesi',
  'road': 'Molla Hüsrev Caddesi',
  'town': 'Fatih',
  'state': 'Marmara Bölgesi',
  'country': 'Türkiye',
  'postcode': '34096',
  'country_code': 'tr'},
 'rate': '1',
 'osm': 'way/331188177',
 'bbox': {'lon_min': 28.949586,
  'lon_max': 28.94984,
  'lat_min': 41.0137,
  'lat_max': 41.01391},
 'kinds': 'religion,other_temples,interesting_places',
 'sources': {'geometry': 'osm', 'attributes': ['osm']},
 'otm': 'https://opentripmap.com/en/card/W331188177',
 'point': {'lon': 28.949715, 'lat': 41.013805}}

### Collecting Places Dataset
**get_place_features from places.py** rreturns the following features for each place in a wanted location:
 1.  `ID`
 2.  `NAME`
 3.  `RATE`
 4.  `Kinds` 
 
a search query to find all places in **Istanbul** for example, the first result is:
`{'id': 'W331188177',
 'name': 'Bıçakçı Alaaddin Cami',
 'rate': 1,
 'kinds': 'religion,other_temples,interesting_places'}`

In [7]:
test = places.get_place_features('istanbul')

In [8]:
test[0]

{'id': 'W331188177',
 'name': 'Bıçakçı Alaaddin Cami',
 'rate': 1,
 'kinds': 'religion,other_temples,interesting_places'}

In [9]:
df = pd.DataFrame(test)

In [10]:
df.head()

,id,name,rate,kinds
0,W331188177,Bıçakçı Alaaddin Cami,1,"religion,other_temples,interesting_places"
1,W523880524,Sofular Molla Hüsrev Camii,1,"religion,other_temples,interesting_places"
2,N4362083790,,0,"urban_environment,cultural,interesting_places,..."
3,W201141056,Kızılminare Cami,1,"religion,other_temples,interesting_places"
4,N5076597921,مطعم المدينة اكسراي,1,"other,unclassified_objects,interesting_places,..."


In [11]:
df.size

560

In [12]:
df.to_csv('places.csv')